<a href="https://colab.research.google.com/github/508101/508101/blob/main/CSE584_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install packages
try:
    from collections import deque
except ImportError:
    !pip install collections

import os
from google.colab import files
from collections import deque

# Step 2: Upload files
corpus_file = "/content/hg38_part.txt"
pattern_files = [f"/content/pat{i}.txt" for i in range(1, 6)]

# Step 3: Implement Trie for Aho-Corasick Algorithm
class TrieNode:
    def __init__(self):
        self.children = {}
        self.fail_link = None
        self.output = []

class AhoCorasick:
    def __init__(self, patterns):
        self.root = TrieNode()
        self.patterns = patterns
        self.build_trie(patterns)
        self.build_failure_links()

    def build_trie(self, patterns):
        # Insert patterns into the trie
        for index, pattern in enumerate(patterns):
            node = self.root
            for char in pattern:
                if char not in node.children:
                    node.children[char] = TrieNode()
                node = node.children[char]
            node.output.append(index + 1)

    def build_failure_links(self):
        # Compute failure links using BFS
        queue = deque()
        for node in self.root.children.values():
            node.fail_link = self.root
            queue.append(node)

        while queue:
            curr = queue.popleft()
            for char, child in curr.children.items():
                queue.append(child)

                fail = curr.fail_link
                while fail and char not in fail.children:
                    fail = fail.fail_link
                child.fail_link = fail.children[char] if fail else self.root

                child.output.extend(child.fail_link.output)

    def search(self, text, strand, patterns):
        # Search for patterns in the text
        results = []
        node = self.root

        for i, char in enumerate(text):
            while node and char not in node.children:
                node = node.fail_link
            if not node:
                node = self.root
                continue

            node = node.children[char]
            for pattern_index in node.output:
                # Adjust `t` to represent 1-based start position
                match_start = i - len(patterns[pattern_index - 1]) + 2
                results.append((match_start, pattern_index, strand))

        return results

def reverse_complement(dna):
    # Compute the reverse complement of a DNA sequence
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    dna = dna.upper()  # Convert to uppercase to avoid KeyError
    return "".join(complement[base] for base in reversed(dna))

def read_file(filename):
    # Read a file and return contents as a list of strings
    with open(filename, 'r') as file:
        return [line.strip().upper() for line in file.readlines()]  # Code runs into error shows not all of the alphatebt are upper class

def write_output(filename, results):
    # Write sorted results to a file
    results.sort()
    with open(filename, 'w') as file:
        for t, p, s in results:
            file.write(f"{t} {p} {s}\n")

def process_patterns(corpus_file, pattern_files):
    # Process each pattern file and search in the corpus
    corpus = read_file(corpus_file)[0]

    for pattern_file in pattern_files:
        patterns = read_file(pattern_file)
        ac = AhoCorasick(patterns)

        # Search forward strand
        results = ac.search(corpus, strand=1, patterns=patterns)

        # Search reverse-complement strand
        rev_corpus = reverse_complement(corpus)
        rev_results = ac.search(rev_corpus, strand=2, patterns=patterns)

        # Adjust reverse strand indices
        rev_length = len(corpus)
        rev_results = [(rev_length - t + 1, p, s) for t, p, s in rev_results]

        # Combine results, sort, and output
        results.extend(rev_results)
        output_filename = pattern_file.replace(".txt", ".out")
        write_output(output_filename, results)
        print(f"Processed {pattern_file} -> Output saved in {output_filename}")

# Step 4: Run Processing
process_patterns(corpus_file, pattern_files)

# Step 5: Download Results
for pattern_file in pattern_files:
    output_filename = pattern_file.replace(".txt", ".out")
    files.download(output_filename)
